In [16]:
import requests
import re
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

city = 'nashville/' #*****change this city to what you want!!!!*****

#just grabbing the first 20 pages
#feel free to make this prettier
url1 = 'https://www.zillow.com/homes/for_sale/'+city
url2 = 'https://www.zillow.com/homes/for_sale/'+city+'/2_p/'
url3 = 'https://www.zillow.com/homes/for_sale/'+city+'/3_p/'
url4 = 'https://www.zillow.com/homes/for_sale/'+city+'/4_p/'
url5 = 'https://www.zillow.com/homes/for_sale/'+city+'/5_p/'
url6 = 'https://www.zillow.com/homes/for_sale/'+city+'/6_p/'
url7 = 'https://www.zillow.com/homes/for_sale/'+city+'/7_p/'
url8 = 'https://www.zillow.com/homes/for_sale/'+city+'/8_p/'
url9 = 'https://www.zillow.com/homes/for_sale/'+city+'/9_p/'
url10 = 'https://www.zillow.com/homes/for_sale/'+city+'/10_p/'

#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    r1 = s.get(url1, headers=req_headers)
    r2 = s.get(url2, headers=req_headers)
    r3 = s.get(url3, headers=req_headers)
    r4 = s.get(url4, headers=req_headers)
    r5 = s.get(url5, headers=req_headers)
    r6 = s.get(url6, headers=req_headers)
    r7 = s.get(url7, headers=req_headers)
    r8 = s.get(url8, headers=req_headers)
    r9 = s.get(url9, headers=req_headers)
    r10 = s.get(url10, headers=req_headers)

    data1 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r1.text).group(1))
    data2 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r2.text).group(1))
    data3 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r3.text).group(1))
    data4 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r4.text).group(1))
    data5 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r5.text).group(1))
    data6 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r6.text).group(1))
    data7 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r7.text).group(1))
    data8 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r8.text).group(1))
    data9 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r9.text).group(1))
    data10 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r10.text).group(1))

data_list = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]
            # ,data11,data12,data13,data14,data15,data16,data17,data18,data19,data20]

df = pd.DataFrame()

def make_frame(frame):
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']:
            frame = frame.append(item, ignore_index=True)
    return frame

df = make_frame(df)
    
#drop cols
# df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

#drop dupes
df = df.drop_duplicates(subset='zpid', keep="last")

#filters
df['zestimate'] = df['zestimate'].fillna(0)
df['best_deal'] = df['unformattedPrice'] - df['zestimate']
df = df.sort_values(by='best_deal',ascending=True)

print('shape:', df.shape)
display(df[['id','address','beds','baths','area','price','zestimate','best_deal','hdpData']].head(20))

shape: (400, 49)


,id,address,beds,baths,area,price,zestimate,best_deal,hdpData
339,2076549113,"1038 Cone Blvd, Nashville, TN 37207",4,3.0,1903,"$459,000",573400,-114400,"{'homeInfo': {'zpid': 2076549113, 'streetAddre..."
223,2100167720,"259 Randy Ct, Smithville, TN 37166",None,None,None,"$19,900",0,19900,"{'homeInfo': {'zpid': 2100167720, 'streetAddre..."
363,108782292,"201 Cherry St, Madison, TN 37115",None,None,None,"$34,900",0,34900,"{'homeInfo': {'zpid': 108782292, 'streetAddres..."
398,108738877,"1253 Sunnymeade Dr, Nashville, TN 37216",None,None,None,"$49,900",0,49900,"{'homeInfo': {'zpid': 108738877, 'streetAddres..."
82,296631072,"751 Dry Creek Rd, Goodlettsville, TN 37072",None,None,None,"$99,000",0,99000,"{'homeInfo': {'zpid': 296631072, 'streetAddres..."
40,108738559,"511 Yale Ave, Madison, TN 37115",None,None,None,"$100,000",0,100000,"{'homeInfo': {'zpid': 108738559, 'streetAddres..."
16,59886178,"270 Tampa Dr APT G20, Nashville, TN 37211",2,2.0,1096,"$145,000",0,145000,"{'homeInfo': {'zpid': 59886178, 'streetAddress..."
280,41124082,"8328 Old Pond Creek Rd, Pegram, TN 37143",3,2.0,700,"$150,000",0,150000,"{'homeInfo': {'zpid': 41124082, 'streetAddress..."
355,59888886,"420 Walton Ln APT I50, Madison, TN 37115",1,1.0,652,"$150,000",0,150000,"{'homeInfo': {'zpid': 59888886, 'streetAddress..."
104,59888771,"323 Forest Park Rd #5-10, Madison, TN 37115",2,2.0,1050,"$163,000",0,163000,"{'homeInfo': {'zpid': 59888771, 'streetAddress..."


# Get ALL THE DAMN DATA

In [23]:
pd.set_option('display.max_columns', None)

#flatten
def flatten_json(nested_json, exclude=['']):
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

#merge frames
A = pd.DataFrame([flatten_json(x) for x in df['hdpData']])
df['id'] = df['id'].astype(int)
A['homeInfo_zpid'] = A['homeInfo_zpid'].astype(int)
df = pd.concat([df,A], axis=1).fillna(0)
print(df.shape)
df.head()

(400, 196)


,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,price,unformattedPrice,address,addressStreet,addressCity,addressState,addressZipcode,isUndisclosedAddress,beds,baths,area,latLong,isZillowOwned,variableData,badgeInfo,hdpData,isSaved,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,info2String,info3String,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,hasOpenHouse,openHouseDescription,openHouseEndDate,openHouseStartDate,lotAreaString,brokerName,best_deal,homeInfo_zpid,homeInfo_streetAddress,homeInfo_zipcode,homeInfo_city,homeInfo_state,homeInfo_latitude,homeInfo_longitude,homeInfo_price,homeInfo_bathrooms,homeInfo_bedrooms,homeInfo_livingArea,homeInfo_homeType,homeInfo_homeStatus,homeInfo_daysOnZillow,homeInfo_isFeatured,homeInfo_shouldHighlight,homeInfo_zestimate,homeInfo_rentZestimate,homeInfo_listing_sub_type_is_openHouse,homeInfo_listing_sub_type_is_FSBA,homeInfo_openHouse,homeInfo_isUnmappable,homeInfo_isPreforeclosureAuction,homeInfo_homeStatusForHDP,homeInfo_priceForHDP,homeInfo_open_house_info_open_house_showing_0_open_house_start,homeInfo_open_house_info_open_house_showing_0_open_house_end,homeInfo_isNonOwnerOccupied,homeInfo_isPremierBuilder,homeInfo_isZillowOwned,homeInfo_currency,homeInfo_country,homeInfo_lotAreaValue,homeInfo_lotAreaUnit,homeInfo_datePriceChanged,homeInfo_priceReduction,homeInfo_priceChange,homeInfo_taxAssessedValue,homeInfo_listing_sub_type_is_comingSoon,homeInfo_unit,homeInfo_open_house_info_open_house_showing_1_open_house_start,homeInfo_open_house_info_open_house_showing_1_open_house_end,homeInfo_listing_sub_type_is_newHome,homeInfo_newConstructionType,homeInfo_videoCount,homeInfo_open_house_info_open_house_showing_2_open_house_start,homeInfo_open_house_info_open_house_showing_2_open_house_end,homeInfo_isRentalWithBasePrice,homeInfo_listing_sub_type_is_forAuction,homeInfo_zpid,homeInfo_streetAddress,homeInfo_zipcode,homeInfo_city,homeInfo_state,homeInfo_latitude,homeInfo_longitude,homeInfo_price,homeInfo_bathrooms,homeInfo_bedrooms,homeInfo_livingArea,homeInfo_homeType,homeInfo_homeStatus,homeInfo_daysOnZillow,homeInfo_isFeatured,homeInfo_shouldHighlight,homeInfo_zestimate,homeInfo_rentZestimate,homeInfo_listing_sub_type_is_openHouse,homeInfo_listing_sub_type_is_FSBA,homeInfo_openHouse,homeInfo_isUnmappable,homeInfo_isPreforeclosureAuction,homeInfo_homeStatusForHDP,homeInfo_priceForHDP,homeInfo_open_house_info_open_house_showing_0_open_house_start,homeInfo_open_house_info_open_house_showing_0_open_house_end,homeInfo_isNonOwnerOccupied,homeInfo_isPremierBuilder,homeInfo_isZillowOwned,homeInfo_currency,homeInfo_country,homeInfo_lotAreaValue,homeInfo_lotAreaUnit,homeInfo_datePriceChanged,homeInfo_priceReduction,homeInfo_priceChange,homeInfo_taxAssessedValue,homeInfo_listing_sub_type_is_comingSoon,homeInfo_unit,homeInfo_open_house_info_open_house_showing_1_open_house_start,homeInfo_open_house_info_open_house_showing_1_open_house_end,homeInfo_listing_sub_type_is_newHome,homeInfo_newConstructionType,homeInfo_videoCount,homeInfo_open_house_info_open_house_showing_2_open_house_start,homeInfo_open_house_info_open_house_showing_2_open_house_end,homeInfo_isRentalWithBasePrice,homeInfo_listing_sub_type_is_forAuction,homeInfo_zpid,homeInfo_streetAddress,homeInfo_zipcode,homeInfo_city,homeInfo_state,homeInfo_latitude,homeInfo_longitude,homeInfo_price,homeInfo_bathrooms,homeInfo_bedrooms,homeInfo_livingArea,homeInfo_homeType,homeInfo_homeStatus,homeInfo_daysOnZillow,homeInfo_isFeatured,homeInfo_shouldHighlight,homeInfo_zestimate,homeInfo_rentZestimate,homeInfo_listing_sub_type_is_openHouse,homeInfo_listing_sub_type_is_FSBA,homeInfo_openHouse,homeInfo_isUnmappable,homeInfo_isPreforeclosureAuction,homeInfo_homeStatusForHDP,homeInfo_priceForHDP,homeInfo_open_house_info_open_house_showing_0_open_house_start,homeInfo_open_house_info_open_house_showing_0_open_house_end,homeInfo_i